Figure 6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from nbodykit.lab import *
from nbodykit import setup_logging, style
from nbodykit.cosmology import Planck15
from scipy.interpolate import InterpolatedUnivariateSpline as ius
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # many annoying dask warnings from bigfile
plt.style.use(style.notebook)
setup_logging()
L = 25.0
V = L**3
nJy = 1.0000e-23 

In [ ]:
def bphi(Ns,Xs):
    #X = sigma8 or Dz
    # Function to compute (centered) FD derivative given some numberdensity tuple (Nm,N0,Np)
    # and corresponding redshift/s8 tuple (zm,z0,zp) in order of decreasing redshift (increasing a)
    # or increasing sigma8
    d1pX1,d1pX2 = Xs[0]-Xs[1],-(Xs[1]-Xs[2])
    reld1pX1,reld1pX2 = d1pX1/(Xs[1]),d1pX2/(Xs[1])
    bphi_hi = 2/reld1pX1 *(Ns[0]/Ns[1] -1) #low on N refers to low g-r, redshifts in dec order
    bphi_lo = 2/reld1pX2 *(Ns[2]/Ns[1] -1) 
    #averaging procedure of Barriera++20
    bphi = (bphi_hi + bphi_lo)/2.0
    return bphi


def bin_mask(pos,binsi,binsj,binsk):
    bin0i,bin1i = binsi
    bin0j,bin1j = binsj
    bin0k,bin1k = binsk
    return (((pos[:,0]>=bin1i) | (pos[:,0]<bin0i) ) |
            ((pos[:,1]>=bin1j) | (pos[:,1]<bin0j) ) |
            ((pos[:,2]>=bin1k) | (pos[:,2]<bin0k) ) )

def jackknife_bphi(position_m,position_0,position_p,
                   Xs, L,L_jk):
    # cut the box into (L/L_jk)**3 subboxes, return mean and variance of 
    # mean number density after dropping once subbox at a time
    V= L**3
    V_jk = V-(L_jk)**3
    N_sub_1d = int(L//L_jk)
    assert V%L_jk**3==0.0
    N_sub = N_sub_1d**3
    bins = np.linspace(0,L,N_sub_1d+1)
    rep_jk = np.zeros((N_sub_1d,N_sub_1d,N_sub_1d))
    for i in range(N_sub_1d):
        for j in range(N_sub_1d):
            for k in range(N_sub_1d):
                mask_m = bin_mask(position_m,bins[i:i+2],bins[j:j+2],bins[k:k+2])
                mask_0 = bin_mask(position_0,bins[i:i+2],bins[j:j+2],bins[k:k+2])
                mask_p = bin_mask(position_p,bins[i:i+2],bins[j:j+2],bins[k:k+2])
                n_m,n_0,n_p = np.sum(mask_m)/V_jk,np.sum(mask_0)/V_jk,np.sum(mask_p)/V_jk
                rep_jk[i,j,k] = bphi((n_m,n_0,n_p),Xs)
                
    mean =  np.mean(rep_jk) #
    var = (N_sub-1)*np.mean((rep_jk-mean)**2) 
    return mean,np.sqrt(var)


In [3]:
tng_path = "/pscratch/sd/j/jsull/fnl/camels_tng/"
astrid_path = "/pscratch/sd/j/jsull/fnl/camels_astrid/"

In [ ]:
# see here https://camels.readthedocs.io/en/latest/description.html
camels_scale_factors = np.loadtxt("/pscratch/sd/j/jsull/fnl/camels_tng/camels_snapshot_scale_factors.txt")
camels_snaps = np.array([ '{:03d}'.format(i) for i in range(len(camels_scale_factors))])
snap_dirlist = np.concatenate([[14,18,24,28],range(32,91,2)]) #not all of the groups have redshift output
avail_zs = (1./camels_scale_factors - 1.)[snap_dirlist]
avail_snaps = camels_snaps[snap_dirlist]
camels_zs_snaps = dict(zip(avail_zs, avail_snaps))

s8s = np.array([0.7, 0.8, 0.9]) 
Dzs = Planck15.scale_independent_growth_factor(avail_zs)

In [ ]:
# example code https://camels.readthedocs.io/en/latest/photometry.html
def get_gmr_tng(z,s8_str="1P_p2_0",path=tng_path,observed=False,att_string='intrinsic'):
    if observed: light = 'flux'
    else: light = 'luminosity'
    #att_string is either attenuated or intrinsic
    # Snapshot number
    snap = camels_zs_snaps[z]
    # Subfind catalog name
    group_catalog = path+s8_str+'/'+f'groups_{snap}.hdf5'
    # Photometric catalog name
    pref = "IllustrisTNG" if path == tng_path else "Astrid"
    photo_catalog = path+pref+'_{0}_photometry.hdf5'.format("1P_p1_0") if (s8_str =="1P_p2_0" ) else path+pref+'_{0}_photometry.hdf5'.format(s8_str)
    # open the catalogue
    dL = Planck15.luminosity_distance(z)*1e6/10 *3.086e18/Planck15.h # dL in cm
    dm_abs = 1.19649518e+40 #4*pi*(10 pc)^2 in cmsq 
    if(observed):
        dm = 4*np.pi*dL**2
    else:
        dm = dm_abs
    MAB = 48.6
    with h5py.File(photo_catalog, "r") as hf:
        # Get the subhalo index
        subhalo_index = np.array(hf[f"snap_{snap}/SubhaloIndex"][:], dtype=int)
        # Get the g-band magnitude
        g_band = -2.5*np.log10(hf[f"snap_{snap}/BC03/photometry/{light}/{att_string}/SLOAN/SDSS.g"][:] / dm) - MAB
        r_band = -2.5*np.log10(hf[f"snap_{snap}/BC03/photometry/{light}/{att_string}/SLOAN/SDSS.r"][:] / dm) - MAB
    
        if(observed):
            g_band = -2.5 * np.log10(hf[f"snap_{snap}/BC03/photometry/{light}/{att_string}/SLOAN/SDSS.g"][:] / (1e9 * nJy)) + 8.9
            r_band = -2.5 * np.log10(hf[f"snap_{snap}/BC03/photometry/{light}/{att_string}/SLOAN/SDSS.r"][:] / (1e9 * nJy)) + 8.9

    # we only need this if want to associate the g-band magnitude with the group properties
    with h5py.File(group_catalog, "r") as hf:
        M_star = hf['Subhalo/SubhaloMassType'][:,4]*1e10 # Stellar masses 
        halo_index = np.array(hf['Subhalo/SubhaloGrNr'][:])
        halo_mass200m = np.array(hf['Group/Group_M_Mean200'])[halo_index][:]*1e10 # Halo masses 
        subhalo_pos = np.array(hf['Subhalo/SubhaloPos'][:])/1e3 # Subhalo positions in Mpc/h
    # Filter stellar masses using the subhalo index
    M_star = M_star[subhalo_index]
    halo_mass200m = halo_mass200m[subhalo_index]
    subhalo_pos = subhalo_pos[subhalo_index]
    gminusr = g_band - r_band #intrinsic
    return gminusr, M_star, halo_mass200m, subhalo_pos, g_band, r_band

In [ ]:
DM = 5*np.log( (Planck15.luminosity_distance(avail_zs[-15])*1e6/Planck15.h / 10.0) )

In [ ]:
def get_all_s8_zs(zs, s8_str="1P_p2_0", path=tng_path, observed=False, att_string='attenuated'):
    return [get_gmr_tng(z,s8_str=s8_str,path=path,observed=observed,att_string=att_string) for z in zs]

def pre_mask(gmrqz):
    return (gmrqz[-1]>-1e10) & (gmrqz[1]>1e8)
def get_mask(gmrq,gmrbin,hmassbin=None,smassbin=None,rbin=None ):
    gmr,smass,hmass,subhalo_pos,g,r = gmrq
    mask = (gmr>gmrbin[0]) & (gmr<=gmrbin[1])
    if smassbin is not None:
        mask = mask & (smass>smassbin[0]) & (smass<=smassbin[1])
    if hmassbin is not None:
        mask = mask & (hmass>hmassbin[0]) & (hmass<=hmassbin[1])
    if rbin is not None:
        mask = mask & (r>rbin[0]) & (r<=rbin[1])

    return mask, subhalo_pos[mask]

def single_selection_bin_nbars(pos_s8mgmrqs,pos_s80gmrqs,pos_s8pgmrqs,s8s,L):
    # this is for a single selection bin
    nbars_SU_m = np.zeros(len(avail_zs)-2)
    nbars_SU_v = np.zeros(len(avail_zs)-2)
    nbars_Dz_m = np.zeros(len(avail_zs)-2)
    nbars_Dz_v = np.zeros(len(avail_zs)-2)
    for i in range(len(avail_zs)-2):
        if i>0 and i<len(avail_zs)-1: #FIXME LAZY
            nbars_SU_m[i], nbars_SU_v[i] = jackknife_bphi(pos_s8mgmrqs[i],pos_s80gmrqs[i],pos_s8pgmrqs[i],s8s,L,5.0)
            nbars_Dz_m[i], nbars_Dz_v[i] = jackknife_bphi(pos_s80gmrqs[i-1],pos_s80gmrqs[i],pos_s80gmrqs[i+1],Dzs[i-1:i+2],L,5.0)
    return nbars_SU_m,nbars_SU_v,nbars_Dz_m,nbars_Dz_v

In [ ]:
s8mgmrqsttt = get_all_s8_zs(avail_zs,s8_str="1P_p2_n1",observed=True,att_string=att_string)


In [ ]:
att_string = 'attenuated'
observed = False

#tng
s8mgmrqs = get_all_s8_zs(avail_zs,s8_str="1P_p2_n1",observed=observed,att_string=att_string)
s80gmrqs = get_all_s8_zs(avail_zs,s8_str="1P_p2_0",observed=observed,att_string=att_string)
s8pgmrqs = get_all_s8_zs(avail_zs,s8_str="1P_p2_1",observed=observed,att_string=att_string)

#astrid
s8mgmrqs_a = get_all_s8_zs(avail_zs,s8_str="1P_p2_n1",path=astrid_path,
                         observed=observed,att_string=att_string)
s80gmrqs_a = get_all_s8_zs(avail_zs,s8_str="1P_p2_0",path=astrid_path,
                         observed=observed,att_string=att_string)
s8pgmrqs_a = get_all_s8_zs(avail_zs,s8_str="1P_p2_1",path=astrid_path,
                         observed=observed,att_string=att_string)


In [438]:
# fix a particular bin to try this, gmr bin and halo mass bin
gmrbin=(0.3,0.5)
hmbin = None#(1e11,5e11)
smbin = None
rbin = (-np.inf,-20.)#None
nbar_s8mgmrqs = [np.sum(( pre_mask(s8mgmrq) & get_mask(s8mgmrq,gmrbin,hmbin,smbin,rbin)[0] ) )/V for i,s8mgmrq in enumerate(s8mgmrqs)]
nbar_s80gmrqs = [np.sum(( pre_mask(s80gmrq) & get_mask(s80gmrq,gmrbin,hmbin,smbin,rbin)[0] ) )/V for i,s80gmrq in enumerate(s80gmrqs)]
nbar_s8pgmrqs = [np.sum(( pre_mask(s8pgmrq) & get_mask(s8pgmrq,gmrbin,hmbin,smbin,rbin)[0] ) )/V for i,s8pgmrq in enumerate(s8pgmrqs)]
#positions
pos_s8mgmrqs = [get_mask(s8mgmrq,gmrbin,hmbin,smbin,rbin)[1] for i,s8mgmrq in enumerate(s8mgmrqs)]
pos_s80gmrqs = [get_mask(s80gmrq,gmrbin,hmbin,smbin,rbin)[1] for i,s80gmrq in enumerate(s80gmrqs)]
pos_s8pgmrqs = [get_mask(s8pgmrq,gmrbin,hmbin,smbin,rbin)[1] for i,s8pgmrq in enumerate(s8pgmrqs)]

In [439]:
# do the same for astrid
a_nbar_s8mgmrqs = [np.sum(( pre_mask(s8mgmrq) & get_mask(s8mgmrq,gmrbin,hmbin,smbin,rbin)[0] ) )/V for i,s8mgmrq in enumerate(s8mgmrqs_a)]
a_nbar_s80gmrqs = [np.sum(( pre_mask(s80gmrq) & get_mask(s80gmrq,gmrbin,hmbin,smbin,rbin)[0] ) )/V for i,s80gmrq in enumerate(s80gmrqs_a)]
a_nbar_s8pgmrqs = [np.sum(( pre_mask(s8pgmrq) & get_mask(s8pgmrq,gmrbin,hmbin,smbin,rbin)[0] ) )/V for i,s8pgmrq in enumerate(s8pgmrqs_a)]

a_pos_s8mgmrqs = [get_mask(s8mgmrq,gmrbin,hmbin,smbin,rbin)[1] for i,s8mgmrq in enumerate(s8mgmrqs_a)]
a_pos_s80gmrqs = [get_mask(s80gmrq,gmrbin,hmbin,smbin,rbin)[1] for i,s80gmrq in enumerate(s80gmrqs_a)]
a_pos_s8pgmrqs = [get_mask(s8pgmrq,gmrbin,hmbin,smbin,rbin)[1] for i,s8pgmrq in enumerate(s8pgmrqs_a)]

In [ ]:
#tng and astrid
tng_nbars_SU_m,tng_nbars_SU_v,tng_nbars_Dz_m,tng_nbars_Dz_v = single_selection_bin_nbars(pos_s8mgmrqs,pos_s80gmrqs,pos_s8pgmrqs,s8s,L)
a_nbars_SU_m,a_nbars_SU_v,a_nbars_Dz_m,a_nbars_Dz_v = single_selection_bin_nbars(a_pos_s8mgmrqs,a_pos_s80gmrqs,a_pos_s8pgmrqs,s8s,L)

In [ ]:
plt.figure(figsize=(7,5))
plt.title("Attenuated RF $(g-r)$ color: {0}".format(gmrbin)
          +(",\n Halo mass: {0} $M_{\odot}/h$".format(hmbin) if hmbin is not None else "")
          +(",\n Stellar mass: {0} $M_{\odot}/h$".format(smbin) if smbin is not None else "") #FIXME IS THIS PER H?
          +",\n $M_r$: {0} ".format(rbin) if rbin is not None else "",
          fontsize=16)
plt.errorbar(avail_zs[1:-1],tng_nbars_SU_m,yerr=tng_nbars_SU_v,capsize=5,capthick=2,fmt='o',label="TNG: "+r'$\sigma_8$')
plt.errorbar(avail_zs[1:-1],tng_nbars_Dz_m,yerr=tng_nbars_Dz_v,capsize=5,capthick=2,fmt='.',label="TNG: "+r'$D(z)$')
plt.errorbar(avail_zs[1:-1]+0.05,a_nbars_SU_m,yerr=a_nbars_SU_v,capsize=5,capthick=2,fmt='s',label="Astrid: "+r'$\sigma_8$')
plt.errorbar(avail_zs[1:-1]+0.05,a_nbars_Dz_m,yerr=a_nbars_Dz_v,capsize=5,capthick=2,fmt='d',label="Astrid: "+r'$D(z)$')
plt.legend(prop={'size': 12})
plt.xlim(0,2.3)
plt.ylim(-10,50)
plt.axhline(ls='--',c='k')
plt.ylabel(r"$b_{\phi}$",fontsize=20)
plt.xlabel(r"$z$",fontsize=20)
save_name = 'att_camels_bphi_gmr_{0}_{1}'.format(gmrbin[0],gmrbin[1])
if hmbin is not None:
    save_name += "_hmass_{0}_{1}".format(hmbin[0],hmbin[1])
if smbin is not None:
    save_name += "_smass_{0}_{1}".format(smbin[0],smbin[1])
if rbin is not None:
    save_name += "_rmag_{0}_{1}".format(rbin[0],rbin[1])
plt.savefig('../plots/'+save_name+'.png')
plt.savefig('../plots/'+save_name+'.pdf')
